In [1]:
from google.colab import files
import io
import pandas as pd
!pip install geopandas
import geopandas
import fiona
import shapely
import ee
import numpy as np
import folium
ee.Authenticate()
ee.Initialize()

     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 6.3 MB 25.2 MB/s 
     |████████████████████████████████| 16.7 MB 287 kB/s 
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yN970QNYzDSbKf1QLXi2D9eJy7Qf_hEqOP_dZ1a7h1U&tc=MNKTfUlkk-CTfNUCiujeO81ZqHsgeCw-umV8dQ80hS0&cc=zYV8g_F_99Qf6CGrRebgRt1bCSvKFiaeTfuNfAqSlqU

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWg690DZkFqufm7pjok7rKdHBYpA14EUrlTsqvAkuiG8Lny3NsxzkRU

Successfully saved authorization token.


In [3]:
geopandas.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
kml1=files.upload()
file1=kml1['boundary.kml']
aoi=geopandas.read_file(io.BytesIO(file1))
for i in range(0,1):
  tmp1 = [i for i in aoi.geometry]
  x,y = tmp1[i].exterior.coords.xy
  cords = np.dstack((x,y)).tolist()
g=ee.Geometry.Polygon(cords)
g_bbox=g.bounds()

Saving boundary.kml to boundary (1).kml


In [20]:
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filter(ee.Filter.calendarRange(2010,2022,'year')).filter(ee.Filter.calendarRange(6,8,'month')).filter(ee.Filter.calendarRange(1,15,'DAY_OF_MONTH'))
image = collection.filterBounds(g_bbox)
image=image.filter(ee.Filter.lt('CLOUD_COVER',10))
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'],'min': 100, 'max': 3000})
print(image.get('DATATAKE_IDENTIFIER').getInfo())
map = folium.Map(location=[63.250,-151.000],zoom_start=7)
folium.TileLayer(tiles=mapid['tile_fetcher'].url_format,attr='Map Data',overlay=True,name='Landsat8',).add_to(map)
map.add_child(folium.LayerControl())

None


In [ ]:
image_select = image.select(['B2','B3','B4'])
export = ee.batch.Export.image.toDrive(image=image_select.median(),scale=30,crs='EPSG:4269',region=g,maxPixels=3000000000,folder="Folder")
export.start()
print(export.status())

{'state': 'READY', 'description': 'myExportImageTask', 'creation_timestamp_ms': 1650760446243, 'update_timestamp_ms': 1650760446243, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'YMXOJRFGNIOLWJMX4A7N3QYW', 'name': 'projects/earthengine-legacy/operations/YMXOJRFGNIOLWJMX4A7N3QYW'}


In [ ]:
print(export.status())

{'state': 'READY', 'description': 'myExportImageTask', 'creation_timestamp_ms': 1650760446884, 'update_timestamp_ms': 1650760446884, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '7M5MT43FSDHSPZW27EPDBMVM', 'name': 'projects/earthengine-legacy/operations/7M5MT43FSDHSPZW27EPDBMVM'}
